## Sentiment Analysis for looting incident in south africa 2021
   1. https://github.com/rafaljanwojcik/Unsupervised-Sentiment-Analysis
   2. https://medium.com/geekculture/how-to-label-unlabeled-tweets-fb701b97ebf
   3. https://github.com/HudaKas
   
   # Adding an auxiliary task
     1. https://www.kaggle.com/mschumacher/adding-an-auxiliary-task

In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import os; os.environ["TF_KERAS"]='1'

In [3]:
from __future__ import print_function

In [4]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [5]:
import os, sys, re
import ast
import string
import pandas as pd
import numpy as np
import pickle

In [60]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /home/abbey/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/abbey/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/abbey/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [61]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import word_tokenize 
from string import punctuation
from gensim.models import KeyedVectors
from nltk.stem import SnowballStemmer
from bs4 import BeautifulSoup
from nltk.stem.porter import PorterStemmer
from sklearn.feature_selection import chi2
import chardet
from collections import Counter
import itertools

In [62]:
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.style as style

In [63]:
# scikit-sklearn
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, log_loss, make_scorer, confusion_matrix
from sklearn.model_selection import train_test_split, cross_val_predict, StratifiedKFold
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.preprocessing import LabelBinarizer, LabelEncoder, OneHotEncoder,MultiLabelBinarizer
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import gensim
import scikitplot.plotters as skplt

In [64]:
# deep learning libraries
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, concatenate, Dense, Activation, Dropout, SpatialDropout1D,GlobalAveragePooling1D
from tensorflow.keras.layers import Flatten, Embedding, Concatenate, LSTM, Bidirectional, GRU, Conv1D, MaxPooling1D, Layer, GlobalMaxPooling1D
from tensorflow.keras.layers import TimeDistributed, BatchNormalization
from tensorflow.keras import initializers,activations,regularizers,constraints
from tensorflow.keras.preprocessing.text import Tokenizer,one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing import text, sequence
from tensorflow.keras.optimizers import *
from tensorflow.keras import utils
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint, Callback,CSVLogger, ReduceLROnPlateau
# This code was tested with TensorFlow v1.4
print("You have TensorFlow version", tf.__version__)

You have TensorFlow version 2.4.1


In [65]:
from tensorflow.keras.regularizers import *
import gc
from tqdm import tqdm

In [66]:
import time
import random
import torch

In [67]:
from tensorflow.keras.utils import *
from sklearn.metrics import classification_report

In [68]:
from tensorflow.keras.layers import Lambda
from sklearn.metrics import classification_report, confusion_matrix
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
import re

In [69]:
from scipy import interp
from itertools import cycle
from sklearn.metrics import roc_curve, auc

## Load all the dataset
  1. looting_dbn.csv
  2. looting_jhb.csv
  3. looting_pta.csv
  4. PhoenixMassacre.csv

In [17]:
root_dir = '/home/abbey/abbey/dataset/looting/'

In [19]:
df_dbn=pd.read_csv(root_dir + 'looting_dbn.csv', sep='\t')
df_dbn.head(3)

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,1418278758692397061,1418146785084813312,2021-07-22 20:36:21 SAST,2021-07-22,20:36:21,200,2356461407,lordmntungwa,Mashobane,NaN,...,"-29.818746,30.836648,30km",NaN,NaN,NaN,NaN,"[{'screen_name': 'Thabane90252110', 'name': 'B...",NaN,NaN,NaN,NaN
1,1418254055164497923,1418254055164497923,2021-07-22 18:58:11 SAST,2021-07-22,18:58:11,200,436864012,xngubane,xolani Ngubane,NaN,...,"-29.818746,30.836648,30km",NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
2,1418219046470774799,1418217292609966083,2021-07-22 16:39:04 SAST,2021-07-22,16:39:04,200,925778389713203201,mfanakagogo13,Mfanakagogo,NaN,...,"-29.818746,30.836648,30km",NaN,NaN,NaN,NaN,"[{'screen_name': 'MrHandsome_ZA', 'name': 'Mr ...",NaN,NaN,NaN,NaN


In [24]:
df_dbn.columns.unique()

Index(['id', 'conversation_id', 'created_at', 'date', 'time', 'timezone',
       'user_id', 'username', 'name', 'place', 'tweet', 'language', 'mentions',
       'urls', 'photos', 'replies_count', 'retweets_count', 'likes_count',
       'hashtags', 'cashtags', 'link', 'retweet', 'quote_url', 'video',
       'thumbnail', 'near', 'geo', 'source', 'user_rt_id', 'user_rt',
       'retweet_id', 'reply_to', 'retweet_date', 'translate', 'trans_src',
       'trans_dest'],
      dtype='object')

In [25]:
print(len(df_dbn))

1787


In [20]:
df_jhb=pd.read_csv(root_dir + 'looting_jhb.csv', sep='\t')
df_jhb.head(3)

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,1418350406690816002,1418141587515793414,2021-07-23 01:21:03 SAST,2021-07-23,01:21:03,200,845922960,balladunas,Sakhile,NaN,...,"-26.214605,28.034477,25km",NaN,NaN,NaN,NaN,"[{'screen_name': 'MatyaniThulile', 'name': 'Th...",NaN,NaN,NaN,NaN
1,1418346984528822272,1418346984528822272,2021-07-23 01:07:27 SAST,2021-07-23,01:07:27,200,709226036467265537,nongalo_scelo,scelo kaSeme,NaN,...,"-26.214605,28.034477,25km",NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
2,1418317310624944129,1418305359643648004,2021-07-22 23:09:32 SAST,2021-07-22,23:09:32,200,513080845,cathiew632,Cathie Walters #StopTheRotVotelaDA,NaN,...,"-26.214605,28.034477,25km",NaN,NaN,NaN,NaN,"[{'screen_name': 'spaceblacked', 'name': 'milk...",NaN,NaN,NaN,NaN


In [26]:
df_jhb.columns.unique()

Index(['id', 'conversation_id', 'created_at', 'date', 'time', 'timezone',
       'user_id', 'username', 'name', 'place', 'tweet', 'language', 'mentions',
       'urls', 'photos', 'replies_count', 'retweets_count', 'likes_count',
       'hashtags', 'cashtags', 'link', 'retweet', 'quote_url', 'video',
       'thumbnail', 'near', 'geo', 'source', 'user_rt_id', 'user_rt',
       'retweet_id', 'reply_to', 'retweet_date', 'translate', 'trans_src',
       'trans_dest'],
      dtype='object')

In [27]:
print(len(df_jhb))

5448


In [21]:
df_pta=pd.read_csv(root_dir + 'looting_pta.csv', sep='\t')
df_pta.head(3)

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,1418332639136649217,1418085887284498435,2021-07-23 00:10:27 SAST,2021-07-23,00:10:27,200,258747343,bizawe2,Biza NDZAƜƎLA Ѫ,NaN,...,"-25.708385,28.217001,30km",NaN,NaN,NaN,NaN,"[{'screen_name': 'GreatBlackOne', 'name': 'Unc...",NaN,NaN,NaN,NaN
1,1418311375059836931,1418311375059836931,2021-07-22 22:45:57 SAST,2021-07-22,22:45:57,200,138337981,selfierunnerza,Dimakatso David Mokwena (Phoyisa Bae),NaN,...,"-25.708385,28.217001,30km",NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
2,1418284566788329472,1418284566788329472,2021-07-22 20:59:25 SAST,2021-07-22,20:59:25,200,1369618241543405572,xayimpiwilliam,William Xayimpi,NaN,...,"-25.708385,28.217001,30km",NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN


In [28]:
df_pta.columns.unique()

Index(['id', 'conversation_id', 'created_at', 'date', 'time', 'timezone',
       'user_id', 'username', 'name', 'place', 'tweet', 'language', 'mentions',
       'urls', 'photos', 'replies_count', 'retweets_count', 'likes_count',
       'hashtags', 'cashtags', 'link', 'retweet', 'quote_url', 'video',
       'thumbnail', 'near', 'geo', 'source', 'user_rt_id', 'user_rt',
       'retweet_id', 'reply_to', 'retweet_date', 'translate', 'trans_src',
       'trans_dest'],
      dtype='object')

In [29]:
print(len(df_pta))

1952


In [22]:
df_phoenix=pd.read_csv(root_dir + 'PhoenixMassacre.csv', sep='\t')
df_phoenix.head(3)

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,1432964608503996419,1432727354049183750,2021-09-01 09:12:40 SAST,2021-09-01,09:12:40,200,1373250253193904131,african28150616,African Child,NaN,...,NaN,NaN,NaN,NaN,NaN,"[{'screen_name': 'ShareenSingh8', 'name': 'Sha...",NaN,NaN,NaN,NaN
1,1432962079904911360,1432962079904911360,2021-09-01 09:02:37 SAST,2021-09-01,09:02:37,200,1204002979973681152,tusko12,Tusko,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
2,1432960220821655557,1432960220821655557,2021-09-01 08:55:14 SAST,2021-09-01,08:55:14,200,1240611053077245955,snekhumalosa,Former Kleva Black🇿🇦 🇿🇦,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN


In [30]:
df_phoenix.columns.unique()

Index(['id', 'conversation_id', 'created_at', 'date', 'time', 'timezone',
       'user_id', 'username', 'name', 'place', 'tweet', 'language', 'mentions',
       'urls', 'photos', 'replies_count', 'retweets_count', 'likes_count',
       'hashtags', 'cashtags', 'link', 'retweet', 'quote_url', 'video',
       'thumbnail', 'near', 'geo', 'source', 'user_rt_id', 'user_rt',
       'retweet_id', 'reply_to', 'retweet_date', 'translate', 'trans_src',
       'trans_dest'],
      dtype='object')

In [31]:
print(len(df_phoenix))

711


## Filter dataframe

In [32]:
def FilterDataframe(df_dbn, df_jhb, df_pta, df_phoenix):
        columns = ["id", "conversation_id", "created_at", "date", "time", "timezone",
       "user_id", "username", "name", "place", "tweet", "language", "mentions",
       "urls", "photos", "replies_count", "retweets_count", "likes_count",
       "hashtags", "cashtags", "link", "retweet", "quote_url", "video",
       "thumbnail", "near", "geo", "source", "user_rt_id", "user_rt",
       "retweet_id", "reply_to", "retweet_date", "translate", "trans_src",
       "trans_dest"]
        data1 = df_dbn[columns] 
        data2 = df_jhb[columns] 
        data3 = df_pta[columns] 
        data4 = df_phoenix[columns] 
        listed = [data1, data2, data3, data4]
        merged = pd.concat(listed)
        return   merged   

In [33]:
all_looting_df = FilterDataframe(df_dbn, df_jhb, df_pta, df_phoenix)
all_looting_df.head()

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,1418278758692397061,1418146785084813312,2021-07-22 20:36:21 SAST,2021-07-22,20:36:21,200,2356461407,lordmntungwa,Mashobane,NaN,...,"-29.818746,30.836648,30km",NaN,NaN,NaN,NaN,"[{'screen_name': 'Thabane90252110', 'name': 'B...",NaN,NaN,NaN,NaN
1,1418254055164497923,1418254055164497923,2021-07-22 18:58:11 SAST,2021-07-22,18:58:11,200,436864012,xngubane,xolani Ngubane,NaN,...,"-29.818746,30.836648,30km",NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
2,1418219046470774799,1418217292609966083,2021-07-22 16:39:04 SAST,2021-07-22,16:39:04,200,925778389713203201,mfanakagogo13,Mfanakagogo,NaN,...,"-29.818746,30.836648,30km",NaN,NaN,NaN,NaN,"[{'screen_name': 'MrHandsome_ZA', 'name': 'Mr ...",NaN,NaN,NaN,NaN
3,1418208848129642499,1418201374844723204,2021-07-22 15:58:33 SAST,2021-07-22,15:58:33,200,131439206,andydontsa,🕊️#RipDrSindi🕊️ Andiswa Dontsa🇿🇦,NaN,...,"-29.818746,30.836648,30km",NaN,NaN,NaN,NaN,"[{'screen_name': 'Penxenxe', 'name': 'The Vill...",NaN,NaN,NaN,NaN
4,1418201959299960832,1418146785084813312,2021-07-22 15:31:10 SAST,2021-07-22,15:31:10,200,2356461407,lordmntungwa,Mashobane,NaN,...,"-29.818746,30.836648,30km",NaN,NaN,NaN,NaN,"[{'screen_name': 'Curtis_peprah', 'name': 'Sol...",NaN,NaN,NaN,NaN


In [34]:
print(len(all_looting_df))

9898


In [35]:
all_looting_df.to_csv(root_dir +'Combinedlootingdata.csv', index=False)

In [36]:
import matplotlib.pyplot as plt
import seaborn as sns

In [56]:
import string
import re

#nltk
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('stopwords')

# #gensim
from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec


#wordcloud 
from wordcloud import WordCloud 

# #Sklearn
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix, plot_confusion_matrix, classification_report
from sklearn.metrics import accuracy_score, recall_score, get_scorer, f1_score,roc_auc_score,precision_score
from sklearn.feature_extraction.text import TfidfVectorizer


from sklearn.cluster import KMeans
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import MultinomialNB
from xgboost import XGBClassifier


#filter warnings
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to /home/abbey/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [40]:
dataset = pd.read_csv(root_dir + 'Combinedlootingdata.csv', encoding='utf-8')#nrows=51000

In [41]:
dataset.head(3)

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,1418278758692397061,1418146785084813312,2021-07-22 20:36:21 SAST,2021-07-22,20:36:21,200,2356461407,lordmntungwa,Mashobane,NaN,...,"-29.818746,30.836648,30km",NaN,NaN,NaN,NaN,"[{'screen_name': 'Thabane90252110', 'name': 'B...",NaN,NaN,NaN,NaN
1,1418254055164497923,1418254055164497923,2021-07-22 18:58:11 SAST,2021-07-22,18:58:11,200,436864012,xngubane,xolani Ngubane,NaN,...,"-29.818746,30.836648,30km",NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
2,1418219046470774799,1418217292609966083,2021-07-22 16:39:04 SAST,2021-07-22,16:39:04,200,925778389713203201,mfanakagogo13,Mfanakagogo,NaN,...,"-29.818746,30.836648,30km",NaN,NaN,NaN,NaN,"[{'screen_name': 'MrHandsome_ZA', 'name': 'Mr ...",NaN,NaN,NaN,NaN


In [42]:
dataset.tail(4)

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
9895,1430030568612409359,1429784600524513281,2021-08-24 06:53:50 SAST,2021-08-24,06:53:50,200,605685820,lungsta1zn,Lungelo Hlongwa,NaN,...,NaN,NaN,NaN,NaN,NaN,"[{'screen_name': 'BraMahlatse', 'name': 'Mahla...",NaN,NaN,NaN,NaN
9896,1430027273726570498,1430027273726570498,2021-08-24 06:40:45 SAST,2021-08-24,06:40:45,200,1131390998280638464,manyosi62,Manyosi kaNogwaja ozikhundlakhundla,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
9897,1430017767995023362,1430017767995023362,2021-08-24 06:02:58 SAST,2021-08-24,06:02:58,200,1275123464425410560,kgopotsolangeni,Bokone Mokone,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
9898,1430005281531039746,1430005281531039746,2021-08-24 05:13:21 SAST,2021-08-24,05:13:21,200,481157477,dexteraps,#JanikaMallo R.I.P🖤,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN


In [48]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9899 entries, 0 to 9898
Data columns (total 36 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               9899 non-null   object 
 1   conversation_id  9899 non-null   object 
 2   created_at       9899 non-null   object 
 3   date             9899 non-null   object 
 4   time             9899 non-null   object 
 5   timezone         9899 non-null   int64  
 6   user_id          9899 non-null   int64  
 7   username         9899 non-null   object 
 8   name             9899 non-null   object 
 9   place            48 non-null     object 
 10  tweet            9899 non-null   object 
 11  language         9898 non-null   object 
 12  mentions         9897 non-null   object 
 13  urls             9898 non-null   object 
 14  photos           9898 non-null   object 
 15  replies_count    9897 non-null   float64
 16  retweets_count   9898 non-null   object 
 17  likes_count   

In [49]:
cleaned_dataset  = dataset[dataset['tweet'].isnull()==False] 
cleaned_dataset .dropna(subset=['tweet'], inplace=True)

In [50]:
# check for duplicate tweet
cleaned_dataset["tweet"].duplicated(keep="first").sum()

24

In [51]:
# dropping all the duplicates tweets and keeping only the first values
cleaned_dataset = cleaned_dataset.drop_duplicates(subset="tweet",keep="first")

In [52]:
print(len(cleaned_dataset))

9875


In [70]:
#initializing Lemmatizer and stopwords
lemma = WordNetLemmatizer()
stop_words = stopwords.words("english")

In [71]:
# define a function to clean the tweet.
def clean_tweet(tweet):
    tweet = tweet.lower()
    tweet = re.sub('https?:\/\/[a-zA-Z0-9@:%._\/+~#=?&;-]*', ' ', tweet)
    tweet = re.sub('\$[a-zA-Z0-9]*', ' ', tweet)
    tweet = re.sub('\@[a-zA-Z0-9]*', ' ', tweet)
    tweet = re.sub('[^a-zA-Z\']', ' ', tweet)
    
    tweet = ' '.join( [w for w in tweet.split() if len(w)>1] )
    # print(tweet)
    
    tweet=' '.join([lemma.lemmatize(x) for x in nltk.wordpunct_tokenize(tweet) if x not in stop_words])
    tweet=[lemma.lemmatize(x,nltk.corpus.reader.wordnet.VERB) for x in nltk.wordpunct_tokenize(tweet) if x not in stop_words]
    return tweet 

In [72]:
# define a function to clean the Hashtags.
def clean_hashtags(hashtags):
    '''
    hashtags: String
              Input Data
    hashtags: String
              Output Data
           
    func: Convert hashtags to lower case
          Replace ticker symbols with space. The ticker symbols are any stock symbol that starts with $.
          Replace everything not a letter or apostrophe with space
          Removes any spaces or specified characters at the start and end of hashtags.
          
    '''
    if hashtags:
        hashtags = hashtags.lower()
        hashtags = re.sub('\$[a-zA-Z0-9]*', ' ', hashtags)
        hashtags = re.sub('[^a-zA-Z]', ' ', hashtags)
        hashtags=hashtags.strip() 
    return hashtags

In [74]:
# clean the tweets and create two columns: tokenized tweet and whole tweet
cleaned_dataset["clean_tweet"]=cleaned_dataset["tweet"].apply(lambda x: clean_tweet(x))
cleaned_dataset["cleaned_tweet"]=cleaned_dataset["clean_tweet"].apply(lambda x:' '.join(x))

In [75]:
cleaned_dataset.head(2)

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,...,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest,clean_tweet,cleaned_tweet
0,1418278758692397061,1418146785084813312,2021-07-22 20:36:21 SAST,2021-07-22,20:36:21,200,2356461407,lordmntungwa,Mashobane,NaN,...,NaN,NaN,NaN,"[{'screen_name': 'Thabane90252110', 'name': 'B...",NaN,NaN,NaN,NaN,"[peprah, mbalu, fc]",peprah mbalu fc
1,1418254055164497923,1418254055164497923,2021-07-22 18:58:11 SAST,2021-07-22,18:58:11,200,436864012,xngubane,xolani Ngubane,NaN,...,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,"[realy, mean, say, jacob, zuma, go, arrest, st...",realy mean say jacob zuma go arrest stage jaco...


In [76]:
# dataset  = dataset [dataset['cleaned_tweet'].isnull()==False] 
cleaned_dataset= cleaned_dataset[cleaned_dataset["cleaned_tweet"].apply(lambda x: len(x))> 0]

In [77]:
cleaned_dataset.head(2)

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,...,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest,clean_tweet,cleaned_tweet
0,1418278758692397061,1418146785084813312,2021-07-22 20:36:21 SAST,2021-07-22,20:36:21,200,2356461407,lordmntungwa,Mashobane,NaN,...,NaN,NaN,NaN,"[{'screen_name': 'Thabane90252110', 'name': 'B...",NaN,NaN,NaN,NaN,"[peprah, mbalu, fc]",peprah mbalu fc
1,1418254055164497923,1418254055164497923,2021-07-22 18:58:11 SAST,2021-07-22,18:58:11,200,436864012,xngubane,xolani Ngubane,NaN,...,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,"[realy, mean, say, jacob, zuma, go, arrest, st...",realy mean say jacob zuma go arrest stage jaco...


In [78]:
print(len(cleaned_dataset))

9864


#### Converting Date column to DateTime Object and creating two new columns:month and year

In [79]:
cleaned_dataset['date'] = pd.to_datetime(cleaned_dataset['date'],dayfirst=True, errors='coerce')
cleaned_dataset['month'] = cleaned_dataset['date'].dt.month
cleaned_dataset['year'] = cleaned_dataset['date'].dt.year

In [80]:
cleaned_dataset.tail()

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,...,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest,clean_tweet,cleaned_tweet,month,year
9894,1430031220675653633,1429784600524513281,2021-08-24 06:56:26 SAST,2021-08-24,06:56:26,200,1000737710498295808,ka_shuku,NtombiKaShuku,NaN,...,NaN,"[{'screen_name': 'BraMahlatse', 'name': 'Mahla...",NaN,NaN,NaN,NaN,"[wish, black, people, energy, phoenixmassacre,...",wish black people energy phoenixmassacre unarm...,8,2021
9895,1430030568612409359,1429784600524513281,2021-08-24 06:53:50 SAST,2021-08-24,06:53:50,200,605685820,lungsta1zn,Lungelo Hlongwa,NaN,...,NaN,"[{'screen_name': 'BraMahlatse', 'name': 'Mahla...",NaN,NaN,NaN,NaN,"[saw, ukuthi, ceo, csa, know, ', consequence, ...",saw ukuthi ceo csa know ' consequence phoenixm...,8,2021
9896,1430027273726570498,1430027273726570498,2021-08-24 06:40:45 SAST,2021-08-24,06:40:45,200,1131390998280638464,manyosi62,Manyosi kaNogwaja ozikhundlakhundla,NaN,...,NaN,[],NaN,NaN,NaN,NaN,"[cradockfour, marikana, lifeesidimeni, xoloben...",cradockfour marikana lifeesidimeni xolobeni ph...,8,2021
9897,1430017767995023362,1430017767995023362,2021-08-24 06:02:58 SAST,2021-08-24,06:02:58,200,1275123464425410560,kgopotsolangeni,Bokone Mokone,NaN,...,NaN,[],NaN,NaN,NaN,NaN,"[still, vote, enemy, country, since, first, ne...",still vote enemy country since first needle sa...,8,2021
9898,1430005281531039746,1430005281531039746,2021-08-24 05:13:21 SAST,2021-08-24,05:13:21,200,481157477,dexteraps,#JanikaMallo R.I.P🖤,NaN,...,NaN,[],NaN,NaN,NaN,NaN,"[date, th, august, pass, marikanamassacre, rem...",date th august pass marikanamassacre remembera...,8,2021


In [86]:
list(cleaned_dataset['cleaned_tweet'][(cleaned_dataset['year']==2021)&(cleaned_dataset['month']==8)][:5])

['sadly end white own medium clearly take side daily maverick never expose phoenix massacre black blame anc',
 'reflection phoenix massacre',
 'remember one plug card phoenixmassacre',
 'da wat abt ramaphosa instigate phoenixmassacre like concomited marikanamassacre',
 'forget another one ooh yeah killer marikana miner inciter instigator phoenix massacre ethnic mobilization utterance support vigilante group']

## Embedding vector

In [89]:
#Converting the "clean_tweet" column in the format supported by embeddings.
sent = [row for row in cleaned_dataset["clean_tweet"]]
#use Gensim Phrases package to automatically detect common phrases (bigrams) from a list of sentences.
phrases = Phrases(sent, min_count=1, progress_per=50000)
bigram = Phraser(phrases)

INFO - 08:02:35: collecting all words and their counts
INFO - 08:02:35: PROGRESS: at sentence #0, processed 0 words and 0 word types
INFO - 08:02:35: collected 95013 token types (unigram + bigrams) from a corpus of 123825 words and 9864 sentences
INFO - 08:02:35: merged Phrases<95013 vocab, min_count=1, threshold=10.0, max_vocab_size=40000000>
INFO - 08:02:35: Phrases lifecycle event {'msg': 'built Phrases<95013 vocab, min_count=1, threshold=10.0, max_vocab_size=40000000> in 0.13s', 'datetime': '2022-02-16T08:02:35.618393', 'gensim': '4.1.2', 'python': '3.9.7 (default, Sep 16 2021, 13:09:58) \n[GCC 7.5.0]', 'platform': 'Linux-4.15.0-142-generic-x86_64-with-glibc2.27', 'event': 'created'}
INFO - 08:02:35: exporting phrases from Phrases<95013 vocab, min_count=1, threshold=10.0, max_vocab_size=40000000>
INFO - 08:02:35: FrozenPhrases lifecycle event {'msg': 'exported FrozenPhrases<4480 phrases, min_count=1, threshold=10.0> from Phrases<95013 vocab, min_count=1, threshold=10.0, max_vocab_s

In [90]:
sentences = bigram[sent]
sentences[2]

['za',
 'truth',
 'zuma',
 'develop',
 'home',
 'province_eastern',
 'cape',
 'still',
 'way',
 'underdevelop',
 'province',
 'produce',
 'president']

In [91]:
#Initializing the word2vec model
import multiprocessing
w2v_model = Word2Vec(min_count=4,window=5,vector_size =300,sample=1e-5, alpha=0.03,min_alpha=0.0007,negative=20,seed= 42,workers=multiprocessing.cpu_count()-1)
#building vocab of the word2vec model from the custom data
w2v_model.build_vocab(sentences, progress_per=50000)

INFO - 08:03:05: Word2Vec lifecycle event {'params': 'Word2Vec(vocab=0, vector_size=300, alpha=0.03)', 'datetime': '2022-02-16T08:03:05.739999', 'gensim': '4.1.2', 'python': '3.9.7 (default, Sep 16 2021, 13:09:58) \n[GCC 7.5.0]', 'platform': 'Linux-4.15.0-142-generic-x86_64-with-glibc2.27', 'event': 'created'}
INFO - 08:03:05: collecting all words and their counts
INFO - 08:03:05: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 08:03:05: collected 17318 word types from a corpus of 108767 raw words and 9864 sentences
INFO - 08:03:05: Creating a fresh vocabulary
INFO - 08:03:05: Word2Vec lifecycle event {'msg': 'effective_min_count=4 retains 3898 unique words (22.508372791315395%% of original 17318, drops 13420)', 'datetime': '2022-02-16T08:03:05.877783', 'gensim': '4.1.2', 'python': '3.9.7 (default, Sep 16 2021, 13:09:58) \n[GCC 7.5.0]', 'platform': 'Linux-4.15.0-142-generic-x86_64-with-glibc2.27', 'event': 'prepare_vocab'}
INFO - 08:03:05: Word2Vec lifecycle ev

## Approach


In [43]:
from time import time 
from collections import defaultdict

import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

In [ ]:
# https://towardsdatascience.com/how-to-create-your-own-hate-tweet-detector-704508c34cd0
# https://towardsdatascience.com/unsupervised-sentiment-analysis-a38bf1906483
def lemmatize(token):
    return WordNetLemmatizer().lemmatize(token, pos='v')

def tokenize(tweet):
    result = []
    for token in gensim.utils.simple_preprocess(tweet):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 2:
            result.append(lemmatize(token))
    res = ' '.join(result)
    return res

def preprocess(tweet):
    result = re.sub(r'(RT\s@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet)
    result = re.sub(r'(@[A-Za-z0-9-_]+)', '', result)
    result = re.sub(r'http\S+', '', result)
    result = re.sub(r'bit.ly/\S+', '', result)
    result = re.sub(r'(.)\1+', r'\1\1', result)
    result = " ".join(re.findall('[A-Z][^A-Z]*', result)) 
    result = re.sub(r'&[\S]+?;', '', result)
    result = re.sub(r'#', ' ', result)
    result = re.sub(r'[^\w\s]', r'', result)    
    result = re.sub(r'\w*\d\w*', r'', result)
    result = re.sub(r'\s\s+', ' ', result)
    result = re.sub(r'(\A\s+|\s+\Z)', '', result)
    result = tokenize(result)
    return result